# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [77]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df.drop([0,9,18,26],0,inplace=True)
    cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)

    l= []
    for i in cities['NHL']:
        i=i.split('[')
        l.append(i[0])
    cities['NHL'] = l

    li = []
    for i in nhl_df['team']:
        i = re.findall("[^*]+", i)
        li.append(i[0])
    nhl_df['team'] = li

    nhl_df = nhl_df.head(31)

    nhl_df['team_ville'] = nhl_df['team']
    nhl_df['team_ville'] = nhl_df['team_ville'].map({'Tampa Bay Lightning':'Tampa Bay Area',
     'Boston Bruins':'Boston',
     'Toronto Maple Leafs':'Toronto',
     'Florida Panthers':'Miami–Fort Lauderdale',
     'Detroit Red Wings':'Detroit',
     'Montreal Canadiens':'Montreal',
     'Ottawa Senators':'Ottawa',
     'Buffalo Sabres':'Buffalo',
     'Washington Capitals':'Washington, D.C.',
     'Pittsburgh Penguins':'Pittsburgh',
     'Philadelphia Flyers':'Philadelphia',
     'Columbus Blue Jackets':'Columbus',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Nashville Predators':'Nashville',
     'Winnipeg Jets':'Winnipeg',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'St. Louis Blues':'St. Louis',
     'Dallas Stars':'Dallas–Fort Worth',
     'Chicago Blackhawks':'Chicago',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'San Jose Sharks':'San Francisco Bay Area',
     'Los Angeles Kings':'Los Angeles',
     'Calgary Flames':'Calgary',
     'Edmonton Oilers':'Edmonton',
     'Vancouver Canucks':'Vancouver',
     'Arizona Coyotes':'Phoenix'})

    df = pd.merge(nhl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

    df['W/L'] = df['W']/(df['L']+df['W'])

    df = df.groupby('Metropolitan area').mean().reset_index()
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv").head(30)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities=cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    cities['NBA'].replace('—', np.nan, inplace=True)
    cities['NBA'] = cities['NBA'].str.replace(r'\[note\s\d+\]$', '', regex=True)
    cities['NBA'].replace('', np.nan, inplace=True)
    cities_NBA = cities.dropna(subset=['NBA'], inplace=False)
    cities_NBA['NBA'].replace('Trail Blazers', 'Blazers', inplace=True)

    cities_NBA['NBA'] = cities_NBA['NBA'].apply(lambda x: x.split())
    cities_NBA = cities_NBA.explode('NBA').reset_index(drop=True)
    nba_df['team'] = nba_df['team'].str.replace(r'\(\d+\)$', '', regex=True)
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split()[-1])
    nba_df['team'] = nba_df['team'].str.replace('*','')
    nba_df =nba_df[['team','W','L']]
    df = pd.merge(cities_NBA, nba_df, how='left', left_on='NBA', right_on='team', suffixes=('_city', '_nhl'))
    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
    df = df[he]
    df['W/L'] = df['W'] / (df['L'] + df['W'])
    df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})

    population_by_region = df['Population (2016 est.)[8]'] 
    win_loss_by_region = df['W/L']
    

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv").head(30)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities=cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    cities['MLB'].replace('—', np.nan, inplace=True)
    cities['MLB'] = cities['MLB'].str.replace(r'\[note\s\d+\]$', '', regex=True)
    cities['MLB'].replace('', np.nan, inplace=True)
    cities_MLB = cities.dropna(subset=['MLB'], inplace=False)
    cities_MLB['MLB'].replace({'Cubs White Sox':'Cubs White','Red Sox':'Red'}, inplace=True)

    cities_MLB['MLB'] = cities_MLB['MLB'].apply(lambda x: x.split())
    cities_MLB = cities_MLB.explode('MLB').reset_index(drop=True)
    mlb_df['team'].replace({'Boston Red Sox':'Boston Red','Chicago White Sox':'Chicago White'},inplace=True)
    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.split()[-1])
    mlb_df =mlb_df[['team','W','L']]
    df = pd.merge(cities_MLB, mlb_df, how='left', left_on='MLB', right_on='team', suffixes=('_city', '_nhl'))
    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
    df = df[he]
    df['W/L'] = df['W'] / (df['L'] + df['W'])
    df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})

    population_by_region = df['Population (2016 est.)[8]'] 
    win_loss_by_region = df['W/L']

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

         Metropolitan area Population (2016 est.)[8]           MLB  \
0            New York City                  20153634       Yankees   
1            New York City                  20153634          Mets   
2              Los Angeles                  13310447       Dodgers   
3              Los Angeles                  13310447        Angels   
4   San Francisco Bay Area                   6657982        Giants   
5   San Francisco Bay Area                   6657982     Athletics   
6                  Chicago                   9512999          Cubs   
7                  Chicago                   9512999         White   
8        Dallas–Fort Worth                   7233323       Rangers   
9         Washington, D.C.                   6131977     Nationals   
10            Philadelphia                   6070500      Phillies   
11                  Boston                   4794447           Red   
12  Minneapolis–Saint Paul                   3551036         Twins   
13                  

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [45]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv").head(40)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df.drop([0, 5, 10, 15,20,25,30,35], axis=0, inplace=True)
    cities=cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    cities['NFL'] = cities['NFL'].str.replace(r'\[note\s\d+\]$', '', regex=True)
    cities['NFL'].replace('—', np.nan, inplace=True)
    cities['NFL'].replace('', np.nan, inplace=True)
    cities_NFL = cities.dropna(subset=['NFL'], inplace=False)
    cities_NFL = cities_NFL[cities_NFL['Metropolitan area'] != 'Toronto']
    cities_NFL['NFL'] = cities_NFL['NFL'].apply(lambda x: x.split())
    cities_NFL = cities_NFL.explode('NFL').reset_index(drop=True)
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split()[-1])
    nfl_df['team'] = nfl_df['team'].str.replace('*','')
    nfl_df['team'] = nfl_df['team'].str.replace('+','')
    nfl_df =nfl_df[['team','W','L']]
    df = pd.merge(cities_NFL, nfl_df, how='left', left_on='NFL', right_on='team', suffixes=('_city', '_nfl'))
    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
    df = df[he]
    df['W/L'] = df['W'] / (df['L'] + df['W'])
    df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})

    population_by_region = df['Population (2016 est.)[8]'] 
    win_loss_by_region = df['W/L']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

         Metropolitan area Population (2016 est.)[8]         NFL        team  \
0            New York City                  20153634      Giants      Giants   
1            New York City                  20153634        Jets        Jets   
2              Los Angeles                  13310447        Rams        Rams   
3              Los Angeles                  13310447    Chargers    Chargers   
4   San Francisco Bay Area                   6657982       49ers       49ers   
5   San Francisco Bay Area                   6657982     Raiders     Raiders   
6                  Chicago                   9512999       Bears       Bears   
7        Dallas–Fort Worth                   7233323     Cowboys     Cowboys   
8         Washington, D.C.                   6131977    Redskins    Redskins   
9             Philadelphia                   6070500      Eagles      Eagles   
10                  Boston                   4794447    Patriots    Patriots   
11  Minneapolis–Saint Paul              

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
def sports_team_performance(): 
    def nhl():
        nhl_df=pd.read_csv("assets/nhl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        #remove anything in brackets
        cities['NHL'] = cities['NHL'].str.replace('\[.+\]','') #.replace('—',np.nan).replace('',np.nan).dropna()

        #split out the teams when there are multiples in a cell
        def split_teams(ini_str):
            # Splitting on UpperCase using re
            res_list = []
            res_list = re.findall('[A-Z].+\s[A-Z].+|[A-Z][a-z]+', ini_str)
            return res_list

        #split teams into arrays
        cities['NHL'] = cities['NHL'].apply(lambda x: split_teams(x))


        #make a list of cities, teams, and populations
        #break arrays into individual rows for each team
        nhl_map_df = pd.DataFrame([], columns = ['City', 'Pop', 'Team']) 
        i = 0
        r = 0

        while i < cities['NHL'].count():
            j=0
            while j < len(cities.loc[i,'NHL']):
                nhl_map_df.loc[r,'Team'] = cities.loc[i,'NHL'][j] # get team(s)
                nhl_map_df.loc[r,'City'] = cities.loc[i,'Metropolitan area'] # get city
                nhl_map_df.loc[r,'Pop'] = cities.loc[i,'Population (2016 est.)[8]'] # get population 
                j+=1
                r+=1
            i+=1

        #split city
        def split_city(ini_str):
            # Splitting on UpperCase using re
            res_list = []
            res_list = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z][a-z]+', ini_str)[0]
            return res_list

        #one word or start two words with a space between or one word and dash or one word and comma
        nhl_map_df['new_city'] = nhl_map_df['City'].apply(lambda x: split_city(x))
        nhl_map_df['city_team'] = nhl_map_df['new_city'] + ' ' + nhl_map_df['Team']


        #drop division rows without team data    
        nhl_df = nhl_df[~nhl_df['team'].str.contains('Division')]

        #remove asterisk from team names
        nhl_df['team'] = nhl_df['team'].str.replace('*','')

        #find win percentage referred to as win:loss ratio in instructions
        nhl_df['w_l_ratio'] = nhl_df['W'].astype(int) / (nhl_df['W'].astype(int) + nhl_df['L'].astype(int))

        #2018 data only
        nhl_df = nhl_df[nhl_df['year']==2018]

        #map the stragglers
        teamDict = {'New York Devils':'New Jersey Devils','San Francisco Sharks':'San Jose Sharks',
                    'Minneapolis Wild':'Minnesota Wild','Denver Avalanche':'Colorado Avalanche',
                    'Miami Panthers':'Florida Panthers','Phoenix Coyotes':'Arizona Coyotes',
                    'St Blues':'St. Louis Blues','Las Vegas Golden Knights':'Vegas Golden Knights',
                    'Raleigh Hurricanes':'Carolina Hurricanes','Los Angeles Ducks':'Anaheim Ducks'
                    ''}

        nhl_map_df['city_team'] = nhl_map_df['city_team'].replace(teamDict)


        #join datasets
        nhl_full_df = nhl_map_df.merge(nhl_df,how='left',left_on='city_team',right_on='team')

        return nhl_full_df.groupby('City').agg({'Pop': 'max','w_l_ratio':'mean'})
    def nba():
        nba_df=pd.read_csv("assets/nba.csv").head(30)
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities=cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
        cities['NBA'].replace('—', np.nan, inplace=True)
        cities['NBA'] = cities['NBA'].str.replace(r'\[note\s\d+\]$', '', regex=True)
        cities['NBA'].replace('', np.nan, inplace=True)
        cities_NBA = cities.dropna(subset=['NBA'], inplace=False)
        cities_NBA['NBA'].replace('Trail Blazers', 'Blazers', inplace=True)

        cities_NBA['NBA'] = cities_NBA['NBA'].apply(lambda x: x.split())
        cities_NBA = cities_NBA.explode('NBA').reset_index(drop=True)
        nba_df['team'] = nba_df['team'].str.replace(r'\(\d+\)$', '', regex=True)
        nba_df['team'] = nba_df['team'].apply(lambda x: x.split()[-1])
        nba_df['team'] = nba_df['team'].str.replace('*','')
        nba_df =nba_df[['team','W','L']]
        df = pd.merge(cities_NBA, nba_df, how='left', left_on='NBA', right_on='team', suffixes=('_city', '_nhl'))
        df['W'] = pd.to_numeric(df['W'])
        df['L'] = pd.to_numeric(df['L'])
        df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
        he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
        df = df[he]
        df['W/L'] = df['W'] / (df['L'] + df['W'])
        df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})
        return df
    def mlb():
        mlb_df=pd.read_csv("assets/mlb.csv").head(30)
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities=cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
        cities['MLB'].replace('—', np.nan, inplace=True)
        cities['MLB'] = cities['MLB'].str.replace(r'\[note\s\d+\]$', '', regex=True)
        cities['MLB'].replace('', np.nan, inplace=True)
        cities_MLB = cities.dropna(subset=['MLB'], inplace=False)
        cities_MLB['MLB'].replace({'Cubs White Sox':'Cubs White','Red Sox':'Red'}, inplace=True)

        cities_MLB['MLB'] = cities_MLB['MLB'].apply(lambda x: x.split())
        cities_MLB = cities_MLB.explode('MLB').reset_index(drop=True)
        mlb_df['team'].replace({'Boston Red Sox':'Boston Red','Chicago White Sox':'Chicago White'},inplace=True)
        mlb_df['team'] = mlb_df['team'].apply(lambda x: x.split()[-1])
        mlb_df =mlb_df[['team','W','L']]
        df = pd.merge(cities_MLB, mlb_df, how='left', left_on='MLB', right_on='team', suffixes=('_city', '_nhl'))
        df['W'] = pd.to_numeric(df['W'])
        df['L'] = pd.to_numeric(df['L'])
        df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
        he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
        df = df[he]
        df['W/L'] = df['W'] / (df['L'] + df['W'])
        df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})
        return df
    def nfl():
        nfl_df=pd.read_csv("assets/nfl.csv").head(40)
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        nfl_df.drop([0, 5, 10, 15,20,25,30,35], axis=0, inplace=True)
        cities=cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
        cities['NFL'] = cities['NFL'].str.replace(r'\[note\s\d+\]$', '', regex=True)
        cities['NFL'].replace('—', np.nan, inplace=True)
        cities['NFL'].replace('', np.nan, inplace=True)
        cities_NFL = cities.dropna(subset=['NFL'], inplace=False)
        cities_NFL = cities_NFL[cities_NFL['Metropolitan area'] != 'Toronto']
        cities_NFL['NFL'] = cities_NFL['NFL'].apply(lambda x: x.split())
        cities_NFL = cities_NFL.explode('NFL').reset_index(drop=True)
        nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split()[-1])
        nfl_df['team'] = nfl_df['team'].str.replace('*','')
        nfl_df['team'] = nfl_df['team'].str.replace('+','')
        nfl_df =nfl_df[['team','W','L']]
        df = pd.merge(cities_NFL, nfl_df, how='left', left_on='NFL', right_on='team', suffixes=('_city', '_nfl'))
        df['W'] = pd.to_numeric(df['W'])
        df['L'] = pd.to_numeric(df['L'])
        df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
        he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
        df = df[he]
        df['W/L'] = df['W'] / (df['L'] + df['W'])
        df = df.groupby('Metropolitan area').agg({'Population (2016 est.)[8]': 'max','W/L':'mean'})
        return df
    def sport_frame(sport):
        if sport == 'NBA':
            return NBA()
        elif sport == 'MLB':
            return MLB()
        elif sport == 'NHL':
            return NHL()
        elif sport == 'NFL':
            return NFL()
        else:
            'error'
    #==================Do the calcs================================

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)


    #loop through all team pairings
    for i in sports:
        for j in sports:
            merge_df = sport_frame(i).merge(sport_frame(j),how='inner',left_on='City',right_on='City')
            p_values.loc[i,j] = stats.ttest_rel(merge_df['w_l_ratio_x'], merge_df['w_l_ratio_y'])[1]
            
    return p_values

                             NFL       NBA       NHL       MLB
Metropolitan area                                             
Boston                  0.687500  0.670732  0.714286  0.666667
Buffalo                 0.375000       NaN  0.357143       NaN
Calgary                      NaN       NaN  0.513889       NaN
Chicago                 0.750000  0.329268  0.458333  0.482769
Columbus                     NaN       NaN  0.600000       NaN
Dallas–Fort Worth       0.625000  0.292683  0.567568  0.413580
Denver                  0.375000  0.560976  0.589041  0.558282
Detroit                 0.375000  0.475610  0.434783  0.395062
Edmonton                     NaN       NaN  0.473684       NaN
Las Vegas                    NaN       NaN  0.680000       NaN
Los Angeles             0.781250  0.469512  0.622895  0.529122
Miami–Fort Lauderdale   0.437500  0.536585  0.594595  0.391304
Minneapolis–Saint Paul  0.533333  0.573171  0.633803  0.481481
Montreal                     NaN       NaN  0.420290   